Lets locate the data we want and then export it into a .csv

In [8]:
import os
import csv
import pandas as pd
import numpy as np
#from io import StringIO

In [9]:
file_path = "Sample_Data"
data_path = "Sample_Data\\s15-3\\s15-3\\_TC2340ms_resX100um_resY100um.t2t"

Format:\
Sample,Scan,Location,AC,x,y,1,2,...,n-1, n\
We are looking for the ".t2t" file

Here is an example of the opened .t2t file as raw data. We need to remove the first few rows and then update their formatting.

In [10]:
with open(data_path, "r") as file:
    content = file.read()

#Some prints so we can help understand the data better
#print(type(content))
#print(len(content))
#print(content)

lets process our input and try to remove the first few rows

In [11]:
#Generate a list of strings of the content string
lines = content.strip().split("\n")
#print(type(lines))
#print(len(lines))

#Remove the first few lines which are not data
del lines[0:4]
#print(lines[0])

# Split each string by commas to create a list of lists
# This process takes a lot of time. 
# perhaps this can be optimized by splitting the array with the string handling
list_of_lists = [s.split(',') for s in lines]

# Convert the list of lists into a NumPy array
numpy_array = np.array(list_of_lists, dtype=np.float32)
#print(numpy_array)

df = pd.DataFrame(numpy_array)

Here is our dataframe, df. It has 1003 columns for XYZ and then the 1000 features.

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,994,995,996,997,998,999,1000,1001,1002,1003
0,-165397.0,618601.0,-605104.0,-0.001,-0.006,-0.013,-0.008,0.028,-0.024,-0.005,...,0.061,0.058,0.034,0.079,0.039,0.057,0.063,0.066,0.060,0.091
1,-165397.0,618601.0,-605104.0,0.001,-0.009,0.003,0.005,0.001,0.002,0.002,...,0.002,-0.009,0.009,0.004,-0.007,0.001,0.003,-0.001,-0.000,0.007
2,-165397.0,618601.0,-184142.0,0.002,-0.007,0.017,-0.002,-0.010,-0.021,-0.004,...,0.023,0.041,0.037,0.022,0.029,0.028,0.046,0.026,0.066,0.008
3,-165397.0,618601.0,-184142.0,0.006,-0.005,0.002,-0.001,-0.001,0.001,0.005,...,0.007,0.001,0.001,0.002,0.003,0.004,0.006,0.000,0.003,0.003
4,-165397.0,618601.0,-439142.0,-0.012,0.016,-0.016,0.004,0.008,-0.017,0.021,...,0.063,0.047,0.072,0.073,0.033,0.056,0.059,0.036,0.058,0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21499,-167398.0,787588.0,-607623.0,-0.000,-0.006,-0.002,0.007,0.001,-0.002,-0.001,...,-0.001,-0.009,-0.003,0.001,-0.007,-0.001,0.002,0.004,-0.011,-0.005
21500,-166399.0,787588.0,-607461.0,0.016,-0.005,-0.012,-0.012,0.013,-0.006,0.012,...,0.065,0.094,0.070,0.085,0.052,0.091,0.068,0.035,0.033,0.071
21501,-166399.0,787588.0,-607461.0,-0.004,0.003,0.000,-0.000,0.001,0.003,-0.001,...,0.009,-0.006,-0.011,0.010,-0.001,-0.002,0.002,0.001,-0.010,-0.011
21502,-165399.0,787588.0,-607740.0,-0.015,0.003,-0.015,0.015,0.012,0.011,0.009,...,0.019,0.024,0.014,0.043,0.071,0.061,0.086,0.050,0.056,0.090


Now we need to add the columns for Sample, Scan, Location, AC  
Sample : Gotten from file name  
Scan : Gotten from file name  
Location : row count  
AC : figure this out later. Counterfeit or Authentic label

In [ ]:
# Extract Sample and Scan from the file path
# This needs to be made robust for using - and _ as separators
path = data_path.split("\\")[-3]
scan = path.split("-")[1]
sample = path.split("-")[0] 
sample = sample[1:]

#print(scan)
#print(sample)

# Add the columns to the dataframe
df.insert(0, 'Sample', sample)
df.insert(1, 'Scan', scan)
df.insert(2, 'Location', range(1, len(df) + 1))
df.insert(3, 'AC', 0)

# Rename the columns after 'AC' to 'X', 'Y', 'Z', and then start counting from 1
df.columns = list(df.columns[:4]) + ['X', 'Y', 'Z'] + list(range(1, len(df.columns) - 6))

df.head()

,Sample,Scan,Location,AC,X,Y,Z,1,2,3,...,992,993,994,995,996,997,998,999,1000,1001
0,15,3,1,0,-165397.0,618601.0,-605104.0,-0.001,-0.006,-0.013,...,0.061,0.058,0.034,0.079,0.039,0.057,0.063,0.066,0.060,0.091
1,15,3,2,0,-165397.0,618601.0,-605104.0,0.001,-0.009,0.003,...,0.002,-0.009,0.009,0.004,-0.007,0.001,0.003,-0.001,-0.000,0.007
2,15,3,3,0,-165397.0,618601.0,-184142.0,0.002,-0.007,0.017,...,0.023,0.041,0.037,0.022,0.029,0.028,0.046,0.026,0.066,0.008
3,15,3,4,0,-165397.0,618601.0,-184142.0,0.006,-0.005,0.002,...,0.007,0.001,0.001,0.002,0.003,0.004,0.006,0.000,0.003,0.003
4,15,3,5,0,-165397.0,618601.0,-439142.0,-0.012,0.016,-0.016,...,0.063,0.047,0.072,0.073,0.033,0.056,0.059,0.036,0.058,0.062


In [14]:
"""
output_folder = "Outputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

df2.to_csv(os.path.join(output_folder, "output.csv"), index=False)
"""
df.to_csv("output.csv", index=False)

This is the good code that does the good stuff

In [83]:
class dataset():
    def __init__(self, folder_path):
        self.data = []
        self.files = []
        # Call the methods below
        self.get_filepaths(folder_path)
        self.t2t_to_df()

    # This method will find all .t2t files in any subfolder of the folder_path.
    # Files containing the string "error" will be ignored.
    def get_filepaths(self, folder_path):
        # Checks subfolders of a path
        for root, dirs, files in os.walk(folder_path):
            # Ignores files containing the string "error"
            if "error" in root:
                continue
            for file in files:
                # Find .t2t files
                if file.endswith(".t2t"):
                    # Print and append the file path to the files list
                    print("Located ", os.path.join(root, file).replace("\\", "\\\\"))
                    self.files.append(os.path.join(root, file).replace("\\", "\\\\"))

    # This method will process the .t2t files and convert them to .csv files.
    def t2t_to_df(self):
        for file_path in self.files:
            with open(file_path, "r") as file:
                content = file.read()
                lines = content.strip().split("\n")
                #print(lines[0], file_path)
                # Remove the first few lines which are not data
                del lines[0:4]
                # Split each string by commas to create a list of lists
                list_of_lists = [s.split(',') for s in lines]
                # Convert the list of lists into a NumPy array
                numpy_array = np.array(list_of_lists, dtype=np.float32)
                # Create a dataframe from the NumPy array
                df = pd.DataFrame(numpy_array)

                # Extract Sample and Scan from the file path
                # Expected input looks like : 
                # folder_path\\sXX-X\\sXX-X\\_TC2340ms_resX100um_resY100um.t2t
                # or
                # folder_path\\sXX_X\\_TC2340ms_resX100um_resY100um.t2t
                
                # Made robust for inconsistent naming conventions
                path = file_path.split("\\")[-3]

                if "-" in path:
                    scan = path.split("-")[1]
                    sample = path.split("-")[0]
                elif "_" in path:
                    scan = path.split("_")[1]
                    sample = path.split("_")[0]
                sample = sample[1:]

                #print(scan)
                #print(sample)

                # Add the columns to the dataframe
                df.insert(0, 'Sample', sample)
                df.insert(1, 'Scan', scan)
                df.insert(2, 'Location', range(1, len(df) + 1))
                df.insert(3, 'AC', 0)

                # Rename the columns after 'AC' to 'X', 'Y', 'Z', and then start counting from 1
                df.columns = list(df.columns[:4]) + ['X', 'Y', 'Z'] + list(range(1, len(df.columns) - 6))

                # Append the dataframe to the data list
                self.data.append(df)
        #Concatenate all the dataframes together and turn the data list into one dataframe (this makes a really big dataframe)
        self.data = pd.concat(self.data, ignore_index = True)
    
    #This member function will save the dataframe to a csv file when called
    def df_to_csv(self):
        self.data.to_csv("output.csv", index=False)          
        

In [31]:
"""
folder_path = "Sample_Data"

# find all .t2t files in any subfolder of the folder_path.
# This needs to be adjusted to ignore files with errors
for root, dirs, files in os.walk(folder_path):
    if "error" in root:
        continue
    for file in files:
        if file.endswith(".t2t"):
            print(os.path.join(root, file))
"""

'\nfolder_path = "Sample_Data"\n\n# find all .t2t files in any subfolder of the folder_path.\n# This needs to be adjusted to ignore files with errors\nfor root, dirs, files in os.walk(folder_path):\n    if "error" in root:\n        continue\n    for file in files:\n        if file.endswith(".t2t"):\n            print(os.path.join(root, file))\n'

In [84]:
folder_path = "Sample_Data"
test = dataset(folder_path=folder_path)

Located  Sample_Data\\s15-3\\s15-3\\_TC2340ms_resX100um_resY100um.t2t
Located  Sample_Data\\s15-5\\_TC2340ms_resX100um_resY100um.t2t
Located  Sample_Data\\s3_2\\_TC2340ms_resX100um_resY100um.t2t


In [85]:
test.data

,Sample,Scan,Location,AC,X,Y,Z,1,2,3,...,992,993,994,995,996,997,998,999,1000,1001
0,15,3,1,0,-165397.0,618601.0,-605104.0,-0.001,-0.006,-0.013,...,0.061,0.058,0.034,0.079,0.039,0.057,0.063,0.066,0.060,0.091
1,15,3,2,0,-165397.0,618601.0,-605104.0,0.001,-0.009,0.003,...,0.002,-0.009,0.009,0.004,-0.007,0.001,0.003,-0.001,-0.000,0.007
2,15,3,3,0,-165397.0,618601.0,-184142.0,0.002,-0.007,0.017,...,0.023,0.041,0.037,0.022,0.029,0.028,0.046,0.026,0.066,0.008
3,15,3,4,0,-165397.0,618601.0,-184142.0,0.006,-0.005,0.002,...,0.007,0.001,0.001,0.002,0.003,0.004,0.006,0.000,0.003,0.003
4,15,3,5,0,-165397.0,618601.0,-439142.0,-0.012,0.016,-0.016,...,0.063,0.047,0.072,0.073,0.033,0.056,0.059,0.036,0.058,0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63005,3,2,19998,0,-178949.0,619711.0,-426473.0,0.005,-0.004,-0.001,...,-0.005,0.004,-0.005,-0.003,0.002,-0.009,-0.010,-0.001,0.004,0.003
63006,3,2,19999,0,-177949.0,619712.0,-426474.0,-0.015,-0.011,0.022,...,0.026,0.000,0.041,0.021,0.014,0.033,0.061,0.037,0.002,0.021
63007,3,2,20000,0,-177949.0,619712.0,-426474.0,-0.002,-0.003,-0.001,...,-0.002,0.011,-0.002,-0.005,0.000,0.000,0.004,-0.001,-0.001,0.006
63008,3,2,20001,0,-177035.0,619712.0,-426473.0,0.016,0.003,-0.023,...,0.008,-0.001,0.019,-0.011,0.018,0.047,0.041,0.026,0.011,0.029
